In [46]:
import requests
from bs4 import BeautifulSoup


import re

def is_chord(string):
    # Regex para detectar acordes com diferentes tipos e notas de baixo, incluindo "7M"
    pattern = r"^(?:[A-G](?:#|b)?)(?:m|maj|min|sus|dim|aug|add|7M)?\d*(?:#|b)?\d*(?:\/[A-G](?:#|b)?)?$"
    
    # Verifica se a string corresponde ao padrão de acorde
    match = re.match(pattern, string)

    # Retorna True se houver correspondência completa, senão False
    return match is not None and match.group(0) == string


def remove_intro_chords(raw: str):
    lines = raw.split('\n')
    out_lines = []
    for l in lines:
        if '[intro]' in l.lower():
            # out_lines.append('[Intro]')
            continue
        out_lines.extend([l])
    return '\n'.join(out_lines)



def remove_tabs(raw: str):
    return re.sub(r"^[EADGBE]\|[^\n]*\n?", "", raw, flags=re.MULTILINE)



def get_raw(url: str):
    response = requests.get(url)
    html_content = response.text

    soup = BeautifulSoup(html_content, "html.parser")

    pre_section = soup.find("pre")

    output = pre_section.text
    
    return output


def get_chords(raw: str):
    chords = []
    has_solo = False

    for line in raw:
        if '-' in line[0]:
            has_solo = True
        elif any(is_chord(x) for x in line):
            chords.extend([x for x in line if is_chord(x)])

    return set(chords), has_solo


def get_lyrics(raw: list[str], trim_intro_chords: bool = True):
    lyrics = []
    
    if trim_intro_chords:
        raw = remove_intro_chords(raw)

    for line in raw.split('\n'):
        if not line:
            lyrics.append('')
            continue
        # print(f'[{any(not is_chord(x.strip()) for x in line.split())}]line -> ', line, '-> ', line.split())
        if any(not is_chord(x.strip()) and x not in list('()') for x in line.split()) and line:
            lyrics.extend([line.strip()])


    return '\n'.join(lyrics).strip()


In [47]:
def chords(url: str):
    raw = get_raw(url)
    chords, has_solo = get_chords(raw)
    return {"chords": chords, "has_solo": has_solo}



def raw(url: str):
    return {"raw": get_raw(url)}



def lyrics(url: str):
    raw = get_raw(url)
    return {"lyrics": get_lyrics(raw)}

In [51]:
c = lyrics('https://www.cifraclub.com.br/isaias-saad/bondade-de-deus/')
print(c['lyrics'])

[Primeira Parte]

Te amo, Deus
Tua graça nunca falha
Todos os dias
Eu estou em Tuas mãos
Desde quando me levanto
Até eu me deitar
Eu cantarei da bondade de Deus

[Refrão]

És fiel em todo tempo
Em todo tempo Tu és tão, tão bom
Com todo fôlego que tenho
Eu cantarei da bondade de Deus

[Segunda Parte]

Tua doce voz
Que me guia em meio ao fogo
Na escuri_dão
Tua presença me conforta
Sei que és meu Pai
Que amigo és
Vivo na bon_dade de Deus

[Refrão]

És fiel em todo tempo
Em todo tempo Tu és tão, tão bom
Com todo fôlego que tenho
Eu cantarei da bondade de Deus

És fiel em todo tempo
Em todo tempo Tu és tão, tão bom
Com todo fôlego que tenho
Eu cantarei da bondade de Deus


[Ponte]

Tua bondade me seguirá
Me seguirá Senhor
Tua bondade me seguirá
Me seguirá Senhor

Eu me rendo a Ti, te dou o meu ser
Entrego tudo a Ti
Tua bondade me seguirá
Me seguirá Senhor

Tua bondade me seguirá
Me seguirá Senhor
Tua bondade me seguirá
Me seguirá Senhor

Eu me rendo a Ti, te dou o meu ser
Entrego tudo a Ti


In [50]:
c = raw('https://www.cifraclub.com.br/isaias-saad/bondade-de-deus/')
print(remove_tabs(c['raw']))

[Intro] C  C4  C  C4

[Primeira Parte]

         C
Te amo, Deus
      F7M/C      C
Tua graça nunca falha
   G/B    Am7
Todos os dias
      F7M/C       G6
Eu estou em Tuas mãos
                   Am7  F7M/C
Desde quando me levanto
        C     G/B  Am7
Até eu me deitar
         F7M/C     G6      C
Eu cantarei da bondade de Deus

[Refrão]

F7M/C                  C
      És fiel em todo tempo
F7M/C                      C        G6
      Em todo tempo Tu és tão, tão bom
F7M/C                      Am7
      Com todo fôlego que tenho
         F7M/C     G6      C
Eu cantarei da bondade de Deus

[Segunda Parte]

          C
Tua doce voz
         F7M/C          C
Que me guia em meio ao fogo
      G/B  Am7
Na escuri_dão
        F7M/C       G6
Tua presença me conforta
                Am7  F7M/C
Sei que és meu Pai
      C   G/B  Am7
Que amigo és
      F7M/C  G       C
Vivo na bon_dade de Deus

[Refrão]

F7M/C                  C
      És fiel em todo tempo
F7M/C                      C        G6
  